In [1]:
# Get raw data
with open('input/15.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [41]:
# Part 1
class Unit(object):
    def __init__(self, ypos, xpos, team):
        self.pos = (ypos, xpos)
        self.team, self.enemy = 'EG'[::[1,-1][team=='G']]
        self.hits = 200
        self.power = 3
        self.nsteps = None
        self.target_dist = None
        self.target_cell = None
        self.target_unit = None
        
    def __repr__(self):
        return f"{self.team}{self.pos} [{self.hits}]"
    
    def __lt__(self, other):
        return (self.pos) < (other.pos)
    
    def endcheck(self):
        return np.all(arena != self.enemy).item()
    
    def update_nsteps(self):
        self.nsteps = {self.pos: (s:=0)}
        while (z:={l:s+1
                   for (i,j),k in self.nsteps.items() if k == s
                   for l in[(i-1,j),(i+1,j),(i,j-1),(i,j+1)] 
                   if l not in self.nsteps and arena[*l]=='.'}):
            s += 1
            self.nsteps.update(z)
            
    def get_target(self):
        if (z:=sorted([[k,(i,j),l]
                       for (i,j),k in sorted(self.nsteps.items())
                       for l in [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]
                       if arena[*l] == self.enemy])):
            self.target_dist, self.target_cell, tloc = z[0]
            self.target_unit = [i for i in units 
                                if i.hits > 0 and i.pos == tloc][0]
        else:
            self.target_dist, self.target_cell, self.target_unit = (None,None,None)
            
    def move(self):
        if self.target_dist:
            self.target_unit.update_nsteps()
            if (z:=sorted([k 
                           for k,v in self.target_unit.nsteps.items() 
                           if v == self.target_dist and self.nsteps.get(k,-1) == 1])):
                arena[*self.pos] = '.'
                arena[*z[0]] = self.team
                self.pos = z[0]
                
    def attack(self):
        if (self.target_dist is not None
            and self.target_dist < 2
            and (z:=sorted([[i.hits, i.pos, i] 
                            for i in units 
                            if i.hits > 0
                               and i.team != self.team 
                               and sum([abs(j-k) for j,k in zip(i.pos, self.pos)])==1]))):
            z[0][2].hits -= min(z[0][2].hits, self.power)
            if z[0][2].hits == 0:
                arena[*z[0][2].pos] = '.'
                z[0][2].pos = ('X','X')

def battle(elf_power=None):
    global arena, units
    arena = np.array([[j for j in i] for i in rawinput.split('\n')])
    units = [Unit(y,x,j) 
             for y,i in enumerate(rawinput.split('\n')) 
             for x,j in enumerate(i) 
             if j in('E', 'G')]
    if elf_power:
        for i in units:
            if i.team == 'E':
                i.power = elf_power
    nrounds = 0
    finished = False
    while not finished:
        units = sorted([i for i in units if i.hits])
        for unit in units:
            if unit.hits:
                if unit.endcheck():
                    finished = True
                    break
                unit.update_nsteps()
                unit.get_target()
                unit.move()
                unit.attack()
        if not finished:
            nrounds += 1
            
    surviving_elves = len([i for i in units if i.team == 'E' and i.hits])
    outcome = nrounds * sum([i.hits for i in units])
    
    return surviving_elves, outcome            
    
battle()[1]

346574

In [59]:
# Part 2
starting_elves = rawinput.count('E')
ptest = 3
while (z:=battle(ptest))[0] < starting_elves:  ptest += 1
z[1]

60864